### Ingest constructors.json file

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
constructors_schema = StructType(fields = [StructField("constructorId", IntegerType(), False),
                                        StructField("constructorRef", StringType(), False),
                                        StructField("name", StringType(), False),
                                        StructField("nationality", StringType(), False),
                                        StructField("url", StringType(), False),]
                             )

In [0]:
constructors_df = spark.read \
    .schema(constructors_schema) \
    .json(f"{raw_folder_path}/{v_file_date}/constructors.json")

##### Drop unwanted columns from the dataframe

In [0]:
constructors_dropped_df = constructors_df.drop(constructors_df['url'])

##### Rename columns and add ingestion date

In [0]:
from pyspark.sql.functions import lit

In [0]:
constructors_renamed_df = constructors_dropped_df.withColumnRenamed("constructorId", "constructor_id") \
                                             .withColumnRenamed("constructorRef", "constructor_ref") \
                                             .withColumn("data_source", lit(v_data_source)) \
                                             .withColumn("file_date", lit(v_file_date))


In [0]:
constructors_final_df = add_ingestion_date(constructors_renamed_df)

##### Write output to parquet file

In [0]:
constructors_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")


In [0]:
dbutils.notebook.exit("Success")